# Building a bad point pressure lift Controller for a district heating network

## BadPointPressureLiftController: Pressure Control at the Worst Point

The `BadPointPressureLiftController` is a custom controller designed for district heating networks modeled with pandapipes. Its main purpose is to maintain a minimum pressure difference at the network's "worst point", the heat exchanger with the lowest pressure difference (German: Schlechtpunktregelung).

### Key Features

- **Automatic Worst Point Detection:** Identifies the heat exchanger with the lowest pressure difference where heat flow is present.
- **Pressure Regulation:** Adjusts the circulation pump's lift and flow pressures to ensure the pressure difference at the worst point meets a specified minimum target.
- **Proportional Control:** Uses a proportional gain to determine the adjustment magnitude based on the deviation from the target pressure difference.
- **Standby Mode:** If no heat flow is detected, the controller switches the pump to a standby mode with minimum lift and flow pressures.
- **Convergence Check:** Determines if the pressure difference is within a specified tolerance of the target, signaling convergence.

### Usage

- **Initialization:** The controller is initialized with the network, pump index, target pressure difference, tolerance, proportional gain, and minimum pressure settings.
- **Integration:** It can be integrated into a simulation loop, automatically adjusting pump pressures at each time step to maintain optimal network operation.

This controller is particularly useful for ensuring reliable and efficient operation in district heating systems, where maintaining a minimum pressure difference at the most critical point is essential for system stability and performance.

In [1]:
from pandapipes.control import BadPointPressureLiftController

## Example Usage of the BadPointPressureLiftController

To demonstrate the usage of the `BadPointPressureLiftController`, we provide an example with a simple test network. The network is initialized using a `initialize_test_net` function, which sets up a district heating system with two heat consumers, a circulation pump, and several pipes and junctions.

The controller is instantiated and added to the network as follows:

```python
net = initialize_test_net()

dp_controller = BadPointPressureLiftController(net)
net.controller.loc[len(net.controller)] = [dp_controller, True, -1, -1, False, False]
```

This function performs the following steps:
- Creates a pandapipes network with water as the working fluid.
- Adds junctions for the pump, pipes, and heat exchangers.
- Installs a circulation pump with specified flow and lift pressures.
- Adds two heat consumers with configurable heat extraction and return temperatures.
- Connects all components with pipes.
- Runs an initial pipeflow calculation.
- Instantiates the `BadPointPressureLiftController` and registers it in the network's controller table.

Once the network is initialized, the controller will automatically regulate the pump pressures during simulation to maintain the minimum pressure difference at the worst point (the heat exchanger with the lowest pressure difference). This ensures reliable operation and helps prevent under-supply at critical points in the network.

In this implementation, a dp_min of 1 bar is used in the Controller.

In [ ]:
import pandapipes as pp
import numpy as np

from pandapipes.control.run_control import run_control

def initialize_test_net(qext_w=np.array([100000, 200000]),
                        return_temperature=np.array([55, 60]),
                        supply_temperature=85,
                        flow_pressure_pump=4,
                        lift_pressure_pump=1.5,
                        pipetype="110/202 PLUS"):

    net = pp.create_empty_network(fluid="water")

    k = 0.1 # roughness defaults to 0.1

    suply_temperature_k = supply_temperature + 273.15
    return_temperature_k = return_temperature + 273.15

    # Junctions for pump
    j1 = pp.create_junction(net, pn_bar=1.05, tfluid_k=suply_temperature_k, name="Junction 1", geodata=(0, 10))
    j2 = pp.create_junction(net, pn_bar=1.05, tfluid_k=suply_temperature_k, name="Junction 2", geodata=(0, 0))

    # Junctions for connection pipes forward line
    j3 = pp.create_junction(net, pn_bar=1.05, tfluid_k=suply_temperature_k, name="Junction 3", geodata=(10, 0))
    j4 = pp.create_junction(net, pn_bar=1.05, tfluid_k=suply_temperature_k, name="Junction 4", geodata=(60, 0))

    # Junctions for heat exchangers
    j5 = pp.create_junction(net, pn_bar=1.05, tfluid_k=suply_temperature_k, name="Junction 5", geodata=(85, 0))
    j6 = pp.create_junction(net, pn_bar=1.05, tfluid_k=suply_temperature_k, name="Junction 6", geodata=(85, 10))

    # Junctions for connection pipes return line
    j7 = pp.create_junction(net, pn_bar=1.05, tfluid_k=suply_temperature_k, name="Junction 7", geodata=(60, 10))
    j8 = pp.create_junction(net, pn_bar=1.05, tfluid_k=suply_temperature_k, name="Junction 8", geodata=(10, 10))

    pump1 = pp.create_circ_pump_const_pressure(net, j1, j2, p_flow_bar=flow_pressure_pump, plift_bar=lift_pressure_pump,
                                               t_flow_k=suply_temperature_k, type="auto", name="pump1")

    pipe1 = pp.create_pipe(net, j2, j3, std_type=pipetype, length_km=0.01, k_mm=k, name="pipe1", sections=5, text_k=283)
    pipe2 = pp.create_pipe(net, j3, j4, std_type=pipetype, length_km=0.05, k_mm=k, name="pipe2", sections=5, text_k=283)
    pipe3 = pp.create_pipe(net, j4, j5, std_type=pipetype, length_km=0.025, k_mm=k, name="pipe3", sections=5, text_k=283)

    heat_consumer1 = pp.create_heat_consumer(net, from_junction=j5, to_junction=j6, loss_coefficient=0, qext_w=qext_w[0],
                                             treturn_k=return_temperature_k[0], name="heat_consumer_1")


    heat_consumer2 = pp.create_heat_consumer(net, from_junction=j4, to_junction=j7, loss_coefficient=0, qext_w=qext_w[1],
                                             treturn_k=return_temperature_k[1], name="heat_consumer_2")

    pipe4 = pp.create_pipe(net, j6, j7, std_type=pipetype, length_km=0.25, k_mm=k, name="pipe4", sections=5, text_k=283)
    pipe5 = pp.create_pipe(net, j7, j8, std_type=pipetype, length_km=0.05, k_mm=k, name="pipe5", sections=5, text_k=283)
    pipe6 = pp.create_pipe(net, j8, j1, std_type=pipetype, length_km=0.01, k_mm=k, name="pipe6", sections=5, text_k=283)

    pp.pipeflow(net, mode="bidirectional", iter=100)

    return net

print("Setting up test network...")
net = initialize_test_net()

dp_controller = BadPointPressureLiftController(net)
net.controller.loc[len(net.controller)] = [dp_controller, True, -1, -1, False, False]

run_control(net, mode="bidirectional", iter=100)


print("Controller converged successfully!")

Setting up test network...

Controller converged successfully!
Controller converged successfully!


## Comparison of Controller Modes

The `BadPointPressureLiftController` supports two operating modes:

1. **`mode='fixed_preturn'` (default)**: Keeps the return pressure constant by adjusting both `p_flow_bar` and `plift_bar` simultaneously.
2. **`mode='fixed_pflow'`**: Keeps the flow pressure constant by adjusting only `plift_bar`. The return pressure may vary in this mode.

Let's compare both modes to understand their behavior and impact on the network.

In [5]:
import pandas as pd

# Test Mode 1: fixed_preturn (keeps return pressure constant)
net1 = initialize_test_net()
controller1 = BadPointPressureLiftController(net1, mode='fixed_preturn')
net1.controller.loc[len(net1.controller)] = [controller1, True, -1, -1, False, False]

# Get initial values
initial_pflow_1 = net1.circ_pump_pressure["p_flow_bar"].iloc[0]
initial_plift_1 = net1.circ_pump_pressure["plift_bar"].iloc[0]
initial_preturn_1 = initial_pflow_1 - initial_plift_1

# Run control (suppress output)
run_control(net1, max_iter=10)

# Get final values
final_pflow_1 = net1.circ_pump_pressure["p_flow_bar"].iloc[0]
final_plift_1 = net1.circ_pump_pressure["plift_bar"].iloc[0]
final_preturn_1 = final_pflow_1 - final_plift_1
final_dp_min_1 = controller1.dp_min

# Test Mode 2: fixed_pflow (keeps flow pressure constant)
net2 = initialize_test_net()
controller2 = BadPointPressureLiftController(net2, mode='fixed_pflow', min_preturn=2.0)
net2.controller.loc[len(net2.controller)] = [controller2, True, -1, -1, False, False]

# Get initial values
initial_pflow_2 = net2.circ_pump_pressure["p_flow_bar"].iloc[0]
initial_plift_2 = net2.circ_pump_pressure["plift_bar"].iloc[0]
initial_preturn_2 = initial_pflow_2 - initial_plift_2

# Run control (suppress output)
run_control(net2, max_iter=10)

# Get final values
final_pflow_2 = net2.circ_pump_pressure["p_flow_bar"].iloc[0]
final_plift_2 = net2.circ_pump_pressure["plift_bar"].iloc[0]
final_preturn_2 = final_pflow_2 - final_plift_2
final_dp_min_2 = controller2.dp_min

# Create comparison table
comparison_data = {
    'Parameter': ['p_flow (initial)', 'p_flow (final)', 'p_flow (change)',
                  'plift (initial)', 'plift (final)', 'plift (change)',
                  'p_return (initial)', 'p_return (final)', 'p_return (change)',
                  'dp_min (final)', 'Converged', 'Iterations'],
    'Mode: fixed_preturn': [
        f"{initial_pflow_1:.4f} bar",
        f"{final_pflow_1:.4f} bar",
        f"{final_pflow_1 - initial_pflow_1:+.4f} bar",
        f"{initial_plift_1:.4f} bar",
        f"{final_plift_1:.4f} bar",
        f"{final_plift_1 - initial_plift_1:+.4f} bar",
        f"{initial_preturn_1:.4f} bar",
        f"{final_preturn_1:.4f} bar",
        f"{final_preturn_1 - initial_preturn_1:+.4f} bar",
        f"{final_dp_min_1:.4f} bar",
        str(controller1.is_converged(net1)),
        str(controller1.iteration)
    ],
    'Mode: fixed_pflow': [
        f"{initial_pflow_2:.4f} bar",
        f"{final_pflow_2:.4f} bar",
        f"{final_pflow_2 - initial_pflow_2:+.4f} bar",
        f"{initial_plift_2:.4f} bar",
        f"{final_plift_2:.4f} bar",
        f"{final_plift_2 - initial_plift_2:+.4f} bar",
        f"{initial_preturn_2:.4f} bar",
        f"{final_preturn_2:.4f} bar",
        f"{final_preturn_2 - initial_preturn_2:+.4f} bar",
        f"{final_dp_min_2:.4f} bar",
        str(controller2.is_converged(net2)),
        str(controller2.iteration)
    ]
}

df_comparison = pd.DataFrame(comparison_data)

# Display the comparison table
df_comparison

,Parameter,Mode: fixed_preturn,Mode: fixed_pflow
0,p_flow (initial),4.0000 bar,4.0000 bar
1,p_flow (final),3.7149 bar,4.0000 bar
2,p_flow (change),-0.2851 bar,+0.0000 bar
3,plift (initial),1.5000 bar,1.5000 bar
4,plift (final),1.2149 bar,1.2149 bar
5,plift (change),-0.2851 bar,-0.2851 bar
6,p_return (initial),2.5000 bar,2.5000 bar
7,p_return (final),2.5000 bar,2.7851 bar
8,p_return (change),+0.0000 bar,+0.2851 bar
9,dp_min (final),1.4828 bar,1.4828 bar


### Analysis of Results

**Mode 1: `fixed_preturn`**
- The return pressure (`p_return`) remains constant throughout the control process
- Both `p_flow_bar` and `plift_bar` are adjusted by the same amount
- This mode is ideal when maintaining a stable return pressure is critical for the system

**Mode 2: `fixed_pflow`**
- The flow pressure (`p_flow_bar`) remains constant
- Only `plift_bar` is adjusted
- The return pressure (`p_return`) will decrease as `plift_bar` increases
- The `min_preturn` parameter prevents the return pressure from falling below a safe minimum

**Which mode to choose?**
- Use `fixed_preturn` (default) when return pressure stability is important for the overall network
- Use `fixed_pflow` when the flow pressure must remain constant (e.g., due to pressure limitations in the supply line)

---

You can now proceed to run time-series simulations or further analyses, and the controller will handle pressure adjustments as needed.

Suggestions for improvements or alternative approaches are appreciated. Please feel free to contribute.